In [1]:
import pandas as pd
import os
from collections import defaultdict

All data found here: https://rkr.statistikbank.dk/statbank5a/SelectVarVal/Define.asp?MainTable=UDB020&PLanguage=0&PXSId=0&wsid=cflastupd

In [2]:
csv_list = ['./Data/CSV/Ejerlejlighed_2004_2017.csv',
           './Data/CSV/Ejerlejlighed_2018_2024.csv',
           './Data/CSV/Fritidshus_2004_2017.csv',
           './Data/CSV/Fritidshus_2018_2024.csv',
           './Data/CSV/Parcel_2004_2017.csv',
           './Data/CSV/Parcel_2018_2024.csv']

output_dir = './Data/Data_cleaned'

In [3]:
# Changing the data into three columns Pris, Kommune, Dato
file_groups = defaultdict(list)

# Getting the first part of the file name so that they can be combined
for csv in csv_list:
    prefix = '_'.join(os.path.basename(csv).split('_')[:-2])
    file_groups[prefix].append(csv)

for prefix, files in file_groups.items():
    combined_df = pd.DataFrame()
    for csv in files:
        df = pd.read_csv(csv, sep=';', encoding='latin1')

        headers = df.columns[1:]
        cities = df.iloc[:, 0]
        data = df.iloc[:, 1:]

        if len(headers) != data.shape[1]:
            raise ValueError(f"Column length mismatch in file {csv}: expected {len(headers)} columns, got {data.shape[1]} columns")

        df_transposed = data.transpose()
        df_transposed.columns = cities
        df_transposed = df_transposed.reset_index()

        df_transposed.columns = ['Dato'] + cities.tolist()

        df_long = pd.melt(df_transposed, id_vars=['Dato'], var_name='Kommune', value_name='Pris')

        # Converting to datetimes
        df_long['Dato'] = pd.to_datetime(df_long['Dato'], format='%YM%m')

        df_long = df_long[['Pris', 'Kommune', 'Dato']]

        combined_df = pd.concat([combined_df, df_long])

    combined_df.reset_index(drop=True, inplace=True)

    output_file = os.path.join(output_dir, f'Cleaned_{prefix}.csv')
    combined_df.to_csv(output_file, index=False, sep=';', encoding='latin1')

    print(f"Data for {prefix} has been combined and saved to {output_file}")

Data for Ejerlejlighed has been combined and saved to ./Data/Data_cleaned\Cleaned_Ejerlejlighed.csv
Data for Fritidshus has been combined and saved to ./Data/Data_cleaned\Cleaned_Fritidshus.csv
Data for Parcel has been combined and saved to ./Data/Data_cleaned\Cleaned_Parcel.csv


In [4]:
# Combining all 6 into one file with the prefix becoming a new column
combined_df = pd.DataFrame()

for csv in csv_list:
    prefix = '_'.join(os.path.basename(csv).split('_')[:-2])

    df = pd.read_csv(csv, sep=';', encoding='latin1')

    headers = df.columns[1:]
    cities = df.iloc[:, 0]
    data = df.iloc[:, 1:]

    if len(headers) != data.shape[1]:
        raise ValueError(f"Column length mismatch in file {csv}: expected {len(headers)} columns, got {data.shape[1]} columns")

    df_transposed = data.transpose()
    df_transposed.columns = cities
    df_transposed = df_transposed.reset_index()
    df_transposed.columns = ['Dato'] + cities.tolist()
    df_long = pd.melt(df_transposed, id_vars=['Dato'], var_name='Kommune', value_name='Pris')
    df_long['Dato'] = pd.to_datetime(df_long['Dato'], format='%YM%m')
    df_long = df_long[['Pris', 'Kommune', 'Dato']]

    # Add the Boligtype column
    df_long['Boligtype'] = prefix

    combined_df = pd.concat([combined_df, df_long], ignore_index=True)

output_file = os.path.join(output_dir, 'Cleaned_All.csv')
combined_df.to_csv(output_file, index=False, sep=';', encoding='latin1')

print(f"All data has been combined and saved to {output_file}")

All data has been combined and saved to ./Data/Data_cleaned\Cleaned_All.csv
